In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
%matplotlib inline

In [2]:
pip install pyforest

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyforest import *
lazy_imports()

['import nltk',
 'import statistics',
 'from sklearn.linear_model import ElasticNet',
 'import gensim',
 'from xlrd import open_workbook',
 'from sklearn.model_selection import GridSearchCV',
 'import numpy as np',
 'import matplotlib as mpl',
 'from sklearn.preprocessing import LabelEncoder',
 'import keras',
 'import matplotlib.pyplot as plt',
 'import plotly.graph_objs as go',
 'import datetime as dt',
 'from sklearn.ensemble import GradientBoostingRegressor',
 'from fbprophet import Prophet',
 'from pathlib import Path',
 'import dash',
 'from sklearn import metrics',
 'from sklearn import svm',
 'import pydot',
 'import torch',
 'import sys',
 'from scipy import stats',
 'import altair as alt',
 'import glob',
 'from sklearn.model_selection import cross_val_score',
 'from sklearn.linear_model import LogisticRegression',
 'from sklearn.model_selection import train_test_split',
 'from sklearn.preprocessing import OneHotEncoder',
 'import pandas as pd',
 'import tqdm',
 'from sklearn

In [4]:
data = pd.read_csv('reverse.csv')

<IPython.core.display.Javascript object>

In [5]:
# Prepare the data
X = data[['Compressive Strength (MPa)']]
y_wc = data['w/c']
y_fc = data['f/c']
y_cf = data['c/f']

In [6]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [7]:
# Split the data for water-cement ratio prediction
X_train_wc, X_test_wc, y_train_wc, y_test_wc = train_test_split(X, y_wc, test_size=0.2, random_state=42)
# Split the data for fine-coarse aggregate ratio prediction
X_train_fc, X_test_fc, y_train_fc, y_test_fc = train_test_split(X, y_fc, test_size=0.2, random_state=42)
# Split the data for cement-fine aggregate ratio prediction
X_train_cf, X_test_cf, y_train_cf, y_test_cf = train_test_split(X, y_cf, test_size=0.2, random_state=42)

In [8]:
# Initialize the Random Forest models
rf_wc = RandomForestRegressor(n_estimators=100, random_state=42)
rf_fc = RandomForestRegressor(n_estimators=100, random_state=42)
rf_cf = RandomForestRegressor(n_estimators=100, random_state=42)

In [9]:
# Train the models
rf_wc.fit(X_train_wc, y_train_wc)
rf_fc.fit(X_train_fc, y_train_fc)
rf_cf.fit(X_train_cf, y_train_cf)

RandomForestRegressor(random_state=42)

In [10]:
# Make predictions and evaluate
wc_predictions = rf_wc.predict(X_test_wc)
fc_predictions = rf_fc.predict(X_test_fc)
cf_predictions = rf_cf.predict(X_test_cf)

In [11]:
# Calculate and print MSE for evaluations
wc_mse = mean_squared_error(y_test_wc, wc_predictions)
fc_mse = mean_squared_error(y_test_fc, fc_predictions)
cf_mse = mean_squared_error(y_test_cf, cf_predictions)
wc_initial_rmse = np.sqrt(mean_squared_error(y_test_wc, wc_predictions))
fc_initial_rmse = np.sqrt(mean_squared_error(y_test_fc, fc_predictions))
cf_initial_rmse = np.sqrt(mean_squared_error(y_test_cf, cf_predictions))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
print(f'Water-Cement Ratio Prediction MSE: {wc_mse}')
print(f'Fine to Coarse Aggregate Ratio Prediction MSE: {fc_mse}')
print(f'Cement to Fine Aggregate Ratio Prediction MSE: {cf_mse}')
print(f'RMSE for water/cement = {wc_initial_rmse}')
print(f'RMSE for fine/coarse = {fc_initial_rmse}')
print(f'RMSE for cement/fine = {cf_initial_rmse}')

Water-Cement Ratio Prediction MSE: 0.003906067806244524
Fine to Coarse Aggregate Ratio Prediction MSE: 0.004550628108655318
Cement to Fine Aggregate Ratio Prediction MSE: 0.001442331894237434
RMSE for water/cement = 0.06249854243296018
RMSE for fine/coarse = 0.0674583435066065
RMSE for cement/fine = 0.037978044897511956


In [13]:
# Function to predict ratios and calculate concrete components based on compressive strength
def predict_and_calculate_ratios(compressive_strength):
    input_data = pd.DataFrame({'Compressive Strength (MPa)': [compressive_strength]})
    predicted_wc = rf_wc.predict(input_data)[0]
    predicted_fc = rf_fc.predict(input_data)[0]
    predicted_cf = rf_cf.predict(input_data)[0]

    # Calculate the components based on the predicted ratios
    cement = 1  # Assume cement as 1 unit
    water = predicted_wc * cement  # water = w/c ratio * cement
    fine_aggregate = cement / predicted_cf  # fine_aggregate = cement / (c/f)
    coarse_aggregate = fine_aggregate / predicted_fc  # coarse_aggregate = fine_aggregate / (f/c)

    return water, cement, fine_aggregate, coarse_aggregate

# Manual input of compressive strength
# compressive_strength = float(input("Enter the compressive strength in MPa: "))
# water, cement, fine_aggregate, coarse_aggregate = predict_and_calculate_ratios(compressive_strength)

# print(f"Calculated Water : Cement : Fine Aggregate : Coarse Aggregate =   {water:.3f},  {cement:.3f},  {fine_aggregate:.3f},  {coarse_aggregate:.3f}")